In [ ]:
#Waiting for prediction order
import socket

# Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Get local machine name
host = socket.gethostname()
port = 9999
msg='none'
# Bind to the port
server_socket.bind((host, port))

server_socket.listen(5)
print("Standby for prediction order")
while True:
    # Establish a connection
    client_socket, addr = server_socket.accept()
    print("Receive prediction order")
    # Receive no more than 1024 bytes
    msg = (client_socket.recv(1024)).decode('utf-8')
    client_socket.close()
    break

if(msg=='MSFT'):
    #Time recording
    import datetime
    startCNN = datetime.datetime.now()

    #Library list
    # Library imports
    import keras
    from sklearn.metrics import mean_squared_error
    from keras.models import Model, Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
    from keras import optimizers, regularizers
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    import yfinance as yf
    from sklearn import preprocessing
    import matplotlib.pyplot as plt
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
    import ta
    #Set random parameter
    np.random.seed(4)
    tf.random.set_seed(4)

    # Get the data for the stock MSFT between '2012-02-01' to '2019-03-05'
    df = yf.download('MSFT','2012-02-01','2019-03-05')
    # Heiken Ashi calculation
    heiken_ashi_df = df.copy()
    heiken_ashi_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    heiken_ashi_df['HA_Open'] = (df['Open'] + df['Close']) / 2
    heiken_ashi_df['HA_High'] = df[['Open', 'Close', 'High']].max(axis=1)
    heiken_ashi_df['HA_Low'] = df[['Open', 'Close', 'Low']].min(axis=1)

    # Additional technical indicators
    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
    df['MACD'] = ta.trend.macd_diff(df['Close'])
    df['MA_10'] = df['Close'].rolling(window=10).mean()

    # Combine all features into one DataFrame
    df = df.join(heiken_ashi_df[['HA_Open', 'HA_High', 'HA_Low', 'HA_Close']])
    df.dropna(inplace=True)

    next_day_closed = df['Close'].shift(-1, axis=0)
    next_day_closed.drop(index=next_day_closed.index[len(next_day_closed)-1], axis=0, inplace=True)
    df.drop(index=df.index[len(df)-1], axis=0, inplace=True)
    date = df.index
    df.reset_index(drop=True, inplace=True)
    df.tail()
    open_values=df['Open'].to_numpy()
    closed_values=df['Close'].to_numpy()
    date_values=date.to_numpy()



    #convert as an array will be used as feature in classifier
    data = df.to_numpy()
    #convert next closed data to array, will be used as label
    next_day_closed_values=next_day_closed.to_numpy()
    # dataset split and number of rows (history) involved in prediction
    test_split = 0.5
    history_points=100

    #normalized data
    data_normaliser = preprocessing.MinMaxScaler()
    #data=data.reshape(-1, 1)
    data_normalised = data_normaliser.fit_transform(data)
    next_day_closed_values = next_day_closed_values.reshape(-1, 1)
    next_day_closed_values_normalised = data_normaliser.fit_transform(next_day_closed_values)

    data_histories_normalised = np.array([data_normalised[i:i + history_points].copy() for i in range(len(data_normalised) - history_points)])
    n = 1558#int(data_histories_normalised.shape[0] * test_split)
    next_day_closed_values_normalised = np.array([next_day_closed_values_normalised[i + history_points].copy() for i in range(len(next_day_closed_values_normalised) - history_points)])
    next_day_closed_values = np.array([next_day_closed_values[i + history_points].copy() for i in range(len(next_day_closed_values) - history_points)])
    open_values = np.array([open_values[i + history_points].copy() for i in range(len(open_values) - history_points)])
    closed_values= np.array([closed_values[i + history_points].copy() for i in range(len(closed_values) - history_points)])
    date_values= np.array([date_values[i + history_points].copy() for i in range(len(date_values) - history_points)])

    # Generate 12x1 images from the rolling windows
    def create_images(data):
        images = []
        for i in range(len(data)):
            window = data[i]
            image = window.T
            #print(image.shape)
            if image.shape == (13, 100):
                images.append(image)
        return np.array(images)
    #

    images = create_images(data_histories_normalised)
    #print(len(images))
    #TRAIN DATA
    data_train = images[:n]#data_histories_normalised[:n]
    y_train = next_day_closed_values_normalised[:n]

    #TEST DATA
    data_test = images[n:]#data_histories_normalised[n:]
    y_test = next_day_closed_values_normalised[n:]
    #print(len(y_test))
    #print(len(data_test))
    # Reshape for CNN
    data_train = data_train.reshape(-1, 13, 1, 1)
    data_test = data_test.reshape(-1, 13, 1, 1)

    unscaled_y_test = next_day_closed_values[n:]
    unscaled_y=next_day_closed_values
    stacking = pd.DataFrame(columns=['date_time','open', 'close','delta_next_day', 'nextclose'])
    stacking['date_time'] = date_values
    stacking['open'] = open_values
    stacking['close'] = closed_values
    stacking['nextclose'] = next_day_closed


    lr=0.0001
    #CNN MODEL
    mse_=0
    nb_prediction=50
    for ii in range(nb_prediction):
        # Define CNN model
        model = Sequential()
        model.add(Conv2D(64, kernel_size=(3, 3), padding='same', input_shape=(100, 13, 1), kernel_regularizer=regularizers.l2(0.01)))
        model.add(BatchNormalization())
        #model.add(LeakyReLU(negative_slope=0.1))  # Use negative_slope instead of alpha
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=(3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01)))
        model.add(BatchNormalization())
        #model.add(LeakyReLU(negative_slope=0.1))  # Use negative_slope instead of alpha
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.3))
        model.add(Conv2D(256, kernel_size=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01)))
        model.add(BatchNormalization())
        #model.add(LeakyReLU(negative_slope=0.1))  # Use negative_slope instead of alpha
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.35))
        model.add(Dense(1, activation='linear'))

        # Compile the model
        nadam = optimizers.Nadam(learning_rate=ii*lr+lr)
        model.compile(optimizer=nadam, loss='mse')

        # Add early stopping and learning rate scheduler
        early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
        #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)

        # Train the model
        data_train=tf.reshape(data_train,(1558,100,13,1))
        model.fit(data_train, y_train, batch_size=64, epochs=25)

        # Test the model
        data_test=tf.reshape(data_test,(90,100,13,1))
        images=tf.reshape(images,(1648,100,13,1))
        # evaluation
        y_test_predicted = model.predict(data_test)
        y_test_predicted = data_normaliser.inverse_transform(y_test_predicted)
        y_predicted = model.predict(images)#data_histories_normalised)
        y_predicted = data_normaliser.inverse_transform(y_predicted)
        unscaled_y_test = np.reshape(unscaled_y_test, (-1, 1))
     #   print(unscaled_y_test.shape)
     #   print(y_test_predicted.shape)

        assert unscaled_y_test.shape == y_test_predicted.shape
        #real_mse = np.mean(np.square(unscaled_y_test - y_test_predicted))
        #mse=real_mse / (np.max(unscaled_y_test) - np.min(unscaled_y_test)) * 100
        #scaled_mse = scaled_mse+real_mse / (np.max(unscaled_y_test) - np.min(unscaled_y_test)) * 100
        #print("Prediction: ",ii)
        #print('MSE:',tf.keras.losses.MSE(tf.expand_dims(srcTF, axis=-1) , tf.expand_dims(tgtTF, axis=-1)) )

        plt.gcf().set_size_inches(22, 15, forward=True)
        start = 0
        end = -1
        #real = plt.plot(unscaled_y_test[start:end], label='real')
        #pred = plt.plot(y_test_predicted[start:end], label='predicted')
        #real = plt.plot(unscaled_y[start:end], label='real')
        #pred = plt.plot(y_predicted[start:end], label='predicted')
        #plt.legend(['Real', 'Predicted'])
        #plt.show()
        #from datetime import datetime
        #model.save(f'basic_model.h5')

        #If next_day_closed_price is higher than today closed price for given threshold, then long or buy
        #If the difference is negative, short or sell, otherwise hold.
        thresh = 0.1
        hold_=0
        long_=1
        short_=2
        strategy=[]
        closed_price_today=[]
        for i in range (len(unscaled_y)):
            closed_price_today_=unscaled_y[i]
            closed_price_today.append(closed_price_today_)
            predicted_closed_price_tomorrow = y_predicted[i]
            delta=predicted_closed_price_tomorrow-closed_price_today_
            if delta > thresh:
                strategy.append(long_)
            elif delta < 0:
                strategy.append(short_)
            else:
                strategy.append(hold_)
        mse=mean_squared_error(next_day_closed_values, y_predicted)
        mse_=mse_+mse
        pt = datetime.datetime.now()-startCNN
        total_seconds = pt.total_seconds()
        print('Step 1. Number_of_prediction: ',(ii+1),' Average_MSE: ',mse_/(ii+1),' Execution_time (s): ',total_seconds," Learning_rate:", ii*lr+lr)
        S='prediction_'+str(ii)
        stacking[S] = strategy
        print(datetime.datetime.now()-startCNN)

    stacking['delta_next_day'] = next_day_closed_values-y_predicted
    stacking['nextclose']=y_predicted#stacking.drop['nextclose'], axis=1, inplace=True)
    stacking.to_csv('CNNs.csv')

    today = plt.plot(next_day_closed_values, label='Tomorrow')
    tomorrow = plt.plot(y_predicted, label='Tomorrow_Predicted')
    plt.legend(['Tomorrow', 'Tomorrow_Predicted_CNNs'])

    plt.show()

    
# Create a socket object
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostname()  # Get local machine name
port = 12345  # Reserve a port for your service

client_socket.connect((host, port))

with open('distCNNs.csv', 'rb') as f:
    print("Sending file...")
    while True:
        data = f.read(1024)
        if not data:
            break
        client_socket.send(data)
    print("File sent successfully")

client_socket.close()

Standby for prediction order
Receive prediction order



C:\Users\USER\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\USER\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[*********************100%***********************]  1 of 1 completed

C:\Users\USER\anaconda3\lib\site-packages\yfinance\utils.py:681: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')





Epoch 1/25

25/25 [==============================] - 9s 210ms/step - loss: 6.3415
Epoch 2/25
25/25 [==============================] - 5s 206ms/step - loss: 3.4480
Epoch 3/25
25/25 [==============================] - 5s 208ms/step - loss: 3.0112
Epoch 4/25
25/25 [==============================] - 5s 211ms/step - loss: 2.7877
Epoch 5/25
25/25 [==============================] - 5s 207ms/step - loss: 2.6987
Epoch 6/25
25/25 [==============================] - 5s 208ms/step - loss: 2.6515
Epoch 7/25
25/25 [==============================] - 5s 212ms/step - loss: 2.6168
Epoch 8/25
25/25 [==============================] - 5s 208ms/step - loss: 2.5925
Epoch 9/25
25/25 [==============================] - 6s 222ms/step - loss: 2.5779
Epoch 10/25
25/25 [==============================] - 6s 220ms/step - loss: 2.5639
Epoch 11/25
25/25 [==============================] - 5s 218ms/step - loss: 2.5558
Epoch 12/25
25/25 [==============================] - 5s 212ms/step - loss: 2.5388
Epoch 13/25
25/25 [==

25/25 [==============================] - 6s 236ms/step - loss: 2.5302
Epoch 10/25
25/25 [==============================] - 6s 250ms/step - loss: 2.5172
Epoch 11/25
25/25 [==============================] - 6s 236ms/step - loss: 2.5020
Epoch 12/25
25/25 [==============================] - 6s 237ms/step - loss: 2.4890
Epoch 13/25
25/25 [==============================] - 6s 233ms/step - loss: 2.4722
Epoch 14/25
25/25 [==============================] - 6s 232ms/step - loss: 2.4567
Epoch 15/25
25/25 [==============================] - 6s 230ms/step - loss: 2.4405
Epoch 16/25
25/25 [==============================] - 6s 232ms/step - loss: 2.4243
Epoch 17/25
25/25 [==============================] - 6s 240ms/step - loss: 2.4083
Epoch 18/25
25/25 [==============================] - 6s 242ms/step - loss: 2.3909
Epoch 19/25
25/25 [==============================] - 6s 239ms/step - loss: 2.3716
Epoch 20/25
25/25 [==============================] - 6s 245ms/step - loss: 2.3543
Epoch 21/25
25/25 [=========

52/52 [==============================] - 1s 27ms/step
Step 1. Number_of_prediction:  7  Average_MSE:  14820.299054084886  Execution_time (s):  1109.552083  Learning_rate: 0.0007000000000000001
0:18:29.552083
Epoch 1/25
25/25 [==============================] - 10s 273ms/step - loss: 85.1882
Epoch 2/25
25/25 [==============================] - 7s 296ms/step - loss: 2.7157
Epoch 3/25
25/25 [==============================] - 7s 277ms/step - loss: 2.6233
Epoch 4/25
25/25 [==============================] - 7s 265ms/step - loss: 2.6081
Epoch 5/25
25/25 [==============================] - 8s 300ms/step - loss: 2.6047
Epoch 6/25
25/25 [==============================] - 8s 323ms/step - loss: 2.5966
Epoch 7/25
25/25 [==============================] - 7s 289ms/step - loss: 2.5925
Epoch 8/25
25/25 [==============================] - 6s 253ms/step - loss: 2.5864
Epoch 9/25
25/25 [==============================] - 7s 263ms/step - loss: 2.5805
Epoch 10/25
25/25 [==============================] - 6s 253ms